## This notebook will help you train a vanilla Point-Cloud AE with the basic architecture we used in our paper.
    (it assumes latent_3d_points is in the PYTHONPATH and the structural losses have been compiled)

In [2]:
import os.path as osp

from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latent_3d_points.src.tf_utils import reset_tf_graph

/home/prashant/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Define Basic Parameters

In [3]:
top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.

experiment_name = 'single_class_ae'
n_pc_points = 2048                              # Number of points per model.
bneck_size = 128                                # Bottleneck-AE size
ae_loss = 'emd'                             # Loss to optimize: 'emd' or 'chamfer'
class_name = input('Give me the class name (e.g. "chair"): ').lower()

Give me the class name (e.g. "chair"): piano


Load Point-Clouds

In [4]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

239 pclouds were loaded. They belong in 1 shape-classes.


Load default training parameters (some of which are listed here). For more details please use print, etc.

    'batch_size': 50   
    
    'denoising': False     (# by default AE is not denoising)

    'learning_rate': 0.0005

    'z_rotate': False      (# randomly rotate models of each batch)
    
    'loss_display_step': 1 (# display loss at end of this many epochs)
    'saver_step': 10       (# how many epochs to save neural-network)

In [5]:
train_dir = create_dir(osp.join(top_out_dir, experiment_name))
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)

In [6]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).
conf.save(osp.join(train_dir, 'configuration'))

Build AE Model.

In [7]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

Building Encoder
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
('encoder_conv_layer_0', 'conv params = ', 256)
('bnorm params = ', 128)
Tensor("single_class_ae_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
('output size:', 131072, '\n')
('encoder_conv_layer_1', 'conv params = ', 8320)
('bnorm params = ', 256)
Tensor("single_class_ae_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
('output size:', 262144, '\n')
('encoder_conv_layer_2', 'conv params = ', 16512)
('bnorm params = ', 256)
Tensor("single_class_ae_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
('output size:', 262144, '\n')
('encoder_conv_layer_3', 'conv params = ', 33024)
('bnorm params = ', 512)
Tensor("single_class_ae_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
('output size:', 524288, '\n')
('encoder_conv_layer_4', 'conv params = ', 32896)
('bnorm params = ', 256)
Tensor("single_class_ae_2/Relu_4:0", shape=(?, 2048, 128), dtype=flo

Train the AE (save output to train_stats.txt) 

In [1]:
all_pc_data.shape

NameError: name 'all_pc_data' is not defined

In [8]:
buf_size = 1 # flush each line
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(all_pc_data, conf, log_file=fout)
fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.0259', 'loss=', '0.209051233')
INFO:tensorflow:../data/single_class_ae/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.0142', 'loss=', '0.155892000')
('Epoch:', '0003', 'training time (minutes)=', '0.0141', 'loss=', '0.128058035')
('Epoch:', '0004', 'training time (minutes)=', '0.0143', 'loss=', '0.111596821')
('Epoch:', '0005', 'training time (minutes)=', '0.0141', 'loss=', '0.097377308')
('Epoch:', '0006', 'training time (minutes)=', '0.0141', 'loss=', '0.091807738')
('Epoch:', '0007', 'training time (minutes)=', '0.0141', 'loss=', '0.084795574')
('Epoch:', '0008', 'training time (minutes)=', '0.0141', 'loss=', '0.079756895')
('Epoch:', '0009', 'training time (minutes)=', '0.0142', 'loss=', '0.077223584')
('Epoch:', '0010', 'training time (minutes)=', '0.0143', 'loss=', '0.076235916')
INFO:tensorflow:../data/single_class_ae/models.ckpt-10 is not in all_model_che

('Epoch:', '0090', 'training time (minutes)=', '0.0142', 'loss=', '0.051272362')
INFO:tensorflow:../data/single_class_ae/models.ckpt-90 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0091', 'training time (minutes)=', '0.0143', 'loss=', '0.053617127')
('Epoch:', '0092', 'training time (minutes)=', '0.0143', 'loss=', '0.049582812')
('Epoch:', '0093', 'training time (minutes)=', '0.0143', 'loss=', '0.051507382')
('Epoch:', '0094', 'training time (minutes)=', '0.0143', 'loss=', '0.049894951')
('Epoch:', '0095', 'training time (minutes)=', '0.0143', 'loss=', '0.051398999')
('Epoch:', '0096', 'training time (minutes)=', '0.0143', 'loss=', '0.050284496')
('Epoch:', '0097', 'training time (minutes)=', '0.0142', 'loss=', '0.049760585')
('Epoch:', '0098', 'training time (minutes)=', '0.0143', 'loss=', '0.049790050')
('Epoch:', '0099', 'training time (minutes)=', '0.0142', 'loss=', '0.050530102')
('Epoch:', '0100', 'training time (minutes)=', '0.0143', 'loss=', '0.04963991

('Epoch:', '0179', 'training time (minutes)=', '0.0148', 'loss=', '0.046263642')
('Epoch:', '0180', 'training time (minutes)=', '0.0144', 'loss=', '0.046097095')
INFO:tensorflow:../data/single_class_ae/models.ckpt-180 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0181', 'training time (minutes)=', '0.0144', 'loss=', '0.047620622')
('Epoch:', '0182', 'training time (minutes)=', '0.0148', 'loss=', '0.048987727')
('Epoch:', '0183', 'training time (minutes)=', '0.0143', 'loss=', '0.046972030')
('Epoch:', '0184', 'training time (minutes)=', '0.0144', 'loss=', '0.046553260')
('Epoch:', '0185', 'training time (minutes)=', '0.0143', 'loss=', '0.047216867')
('Epoch:', '0186', 'training time (minutes)=', '0.0144', 'loss=', '0.046364038')
('Epoch:', '0187', 'training time (minutes)=', '0.0144', 'loss=', '0.045390004')
('Epoch:', '0188', 'training time (minutes)=', '0.0144', 'loss=', '0.046094863')
('Epoch:', '0189', 'training time (minutes)=', '0.0144', 'loss=', '0.0454619

('Epoch:', '0268', 'training time (minutes)=', '0.0142', 'loss=', '0.040912317')
('Epoch:', '0269', 'training time (minutes)=', '0.0142', 'loss=', '0.041809397')
('Epoch:', '0270', 'training time (minutes)=', '0.0142', 'loss=', '0.041204818')
INFO:tensorflow:../data/single_class_ae/models.ckpt-270 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0271', 'training time (minutes)=', '0.0143', 'loss=', '0.041122732')
('Epoch:', '0272', 'training time (minutes)=', '0.0142', 'loss=', '0.040275502')
('Epoch:', '0273', 'training time (minutes)=', '0.0142', 'loss=', '0.040271000')
('Epoch:', '0274', 'training time (minutes)=', '0.0143', 'loss=', '0.039983680')
('Epoch:', '0275', 'training time (minutes)=', '0.0142', 'loss=', '0.040726016')
('Epoch:', '0276', 'training time (minutes)=', '0.0143', 'loss=', '0.040860582')
('Epoch:', '0277', 'training time (minutes)=', '0.0144', 'loss=', '0.042374151')
('Epoch:', '0278', 'training time (minutes)=', '0.0142', 'loss=', '0.0423006

('Epoch:', '0357', 'training time (minutes)=', '0.0143', 'loss=', '0.039971767')
('Epoch:', '0358', 'training time (minutes)=', '0.0143', 'loss=', '0.039000015')
('Epoch:', '0359', 'training time (minutes)=', '0.0143', 'loss=', '0.039894435')
('Epoch:', '0360', 'training time (minutes)=', '0.0142', 'loss=', '0.041130096')
INFO:tensorflow:../data/single_class_ae/models.ckpt-360 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0361', 'training time (minutes)=', '0.0143', 'loss=', '0.039660877')
('Epoch:', '0362', 'training time (minutes)=', '0.0143', 'loss=', '0.039788844')
('Epoch:', '0363', 'training time (minutes)=', '0.0143', 'loss=', '0.039778339')
('Epoch:', '0364', 'training time (minutes)=', '0.0142', 'loss=', '0.040421407')
('Epoch:', '0365', 'training time (minutes)=', '0.0143', 'loss=', '0.040165769')
('Epoch:', '0366', 'training time (minutes)=', '0.0142', 'loss=', '0.040907658')
('Epoch:', '0367', 'training time (minutes)=', '0.0142', 'loss=', '0.0394036

('Epoch:', '0446', 'training time (minutes)=', '0.0142', 'loss=', '0.038273783')
('Epoch:', '0447', 'training time (minutes)=', '0.0142', 'loss=', '0.037562382')
('Epoch:', '0448', 'training time (minutes)=', '0.0142', 'loss=', '0.038003981')
('Epoch:', '0449', 'training time (minutes)=', '0.0142', 'loss=', '0.037677480')
('Epoch:', '0450', 'training time (minutes)=', '0.0142', 'loss=', '0.038412639')
INFO:tensorflow:../data/single_class_ae/models.ckpt-450 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0451', 'training time (minutes)=', '0.0144', 'loss=', '0.038796714')
('Epoch:', '0452', 'training time (minutes)=', '0.0143', 'loss=', '0.037955489')
('Epoch:', '0453', 'training time (minutes)=', '0.0142', 'loss=', '0.037684722')
('Epoch:', '0454', 'training time (minutes)=', '0.0142', 'loss=', '0.038403140')
('Epoch:', '0455', 'training time (minutes)=', '0.0142', 'loss=', '0.038590075')
('Epoch:', '0456', 'training time (minutes)=', '0.0142', 'loss=', '0.0393606

Get some reconstuctions and latent-codes.

In [9]:
feed_pc, feed_model_names, _ = all_pc_data.next_batch(10)
reconstructions = ae.reconstruct(feed_pc)
latent_codes = ae.transform(feed_pc)

In [20]:
recon_arr, recon_loss = reconstructions
recon_arr.shape, recon_arr[0]

((10, 2048, 3), array([[ 0.24808376, -0.07220863, -0.01851774],
        [ 0.121619  , -0.22664535, -0.05162892],
        [-0.050992  , -0.07772793, -0.02552043],
        ...,
        [-0.05316528,  0.02365882, -0.02594247],
        [-0.13948913,  0.01080071, -0.03317543],
        [-0.15718873,  0.12569316,  0.14378455]], dtype=float32))

In [21]:
import numpy.core.records

In [24]:
recon_arr_struct = numpy.core.records.fromarrays(recon_arr.transpose((2,0,1)),
                                                    names='x, y, z',
                                                    formats='f4, f4, f4')
recon_arr_struct.shape, recon_arr_struct

((10, 2048), rec.array([[( 2.48083755e-01, -0.07220863, -0.01851774),
             ( 1.21619001e-01, -0.22664535, -0.05162892),
             (-5.09920046e-02, -0.07772793, -0.02552043), ...,
             (-5.31652793e-02,  0.02365882, -0.02594247),
             (-1.39489129e-01,  0.01080071, -0.03317543),
             (-1.57188728e-01,  0.12569316,  0.14378455)],
            [( 1.25335515e-01, -0.20750852,  0.015026  ),
             ( 3.57898045e-03, -0.20135814, -0.16801819),
             (-1.21016845e-01, -0.14696896, -0.14816204), ...,
             (-6.15503825e-02,  0.03629451, -0.31195217),
             (-1.22578651e-01,  0.05095785,  0.03149545),
             (-1.12119175e-01,  0.12648435,  0.0948585 )],
            [( 1.40655875e-01, -0.21427195,  0.07564928),
             (-5.38663976e-02, -0.18669136, -0.0958605 ),
             (-1.37459427e-01, -0.10235959, -0.14274377), ...,
             (-8.28693062e-02,  0.00497299, -0.25441524),
             (-1.59451723e-01,  0.0819585 ,

In [28]:
import pathlib
results_dir = 'results'
pathlib.Path(results_dir).mkdir(parents=True, exist_ok=True) 

from latent_3d_points.external.python_plyfile import plyfile
el = [plyfile.PlyElement.describe(recon_arr_struct[i], 'vertex') for i in range(10)]
for i in range(10):
    plyfile.PlyData([el[i]], text=True).write(results_dir+'/test_ply%d.ply' % i)

In [31]:
feed_pc_struct = numpy.core.records.fromarrays(feed_pc.transpose((2,0,1)),
                                                    names='x, y, z',
                                                    formats='f4, f4, f4')
feed_pc_struct.shape, feed_pc_struct

((10, 2048), rec.array([[( 0.16341881,  0.27751648, -0.30591258),
             ( 0.1530696 ,  0.27230483, -0.3051154 ),
             ( 0.15488185,  0.278457  , -0.3038634 ), ...,
             (-0.02617915, -0.24882269,  0.3030783 ),
             (-0.02030906, -0.26313043,  0.3039847 ),
             (-0.04632105, -0.25781667,  0.30591258)],
            [(-0.11423104,  0.04121629, -0.30969796),
             (-0.11328048, -0.16444261, -0.30969784),
             (-0.11247163, -0.20731589, -0.30969775), ...,
             ( 0.02860039,  0.24116069,  0.309697  ),
             ( 0.0389767 ,  0.27935264,  0.309697  ),
             ( 0.03815221,  0.2881268 ,  0.309697  )],
            [(-0.18028238,  0.3179567 , -0.28394315),
             (-0.1268053 ,  0.32522163, -0.2838938 ),
             ( 0.02967381,  0.34388316, -0.28359804), ...,
             ( 0.0489579 , -0.34029528,  0.28178164),
             ( 0.04090161, -0.34169525,  0.2820895 ),
             (-0.02727016,  0.34227726,  0.28394315)]

In [32]:
feed_el = [plyfile.PlyElement.describe(feed_pc_struct[i], 'vertex') for i in range(10)]
for i in range(10):
    plyfile.PlyData([feed_el[i]], text=True).write(results_dir+'/train_ply%d.ply' % i)